In [6]:
from datetime import datetime
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time
# from torch.cuda.amp import autocast
import numpy as np
from CVUSA_dataset import CVUSA_dataset_cropped, CVUSA_Dataset_Eval
# from CVUSA_dataset import CVUSA_Dataset_Eval
from custom_models import ResNet, VIT, CLIP_model
from losses import Contrastive_loss, SoftTripletBiLoss, InfoNCE
from train import train
from eval import predict, accuracy, calculate_scores
import torch.nn.functional as F
import copy
import math
from pytorch_metric_learning import losses as LS
from helper_func import get_rand_id, hyparam_info, save_losses
from transformers import CLIPProcessor




# data_path = '/media/fahimul/2B721C03261BDC8D/Research/datasets/CVUSA' #don't include the / at the end
# data_path = '/home/fa947945/datasets/CVUSA_Cropped/CVUSA' #don't include the / at the end
data_path = '/data/Research/Dataset/CVUSA_Cropped/CVUSA' #don't include the / at the end

# train_data= pd.read_csv(f'{data_path}/splits/train-19zl.csv')
train_data= pd.read_csv(f'{data_path}/splits/train-19zl_30.csv')
val_data= pd.read_csv(f'{data_path}/splits/val-19zl.csv')

# df_loss = pd.DataFrame(columns=['Loss'])

transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225]),
])


train_ds = CVUSA_dataset_cropped(df = train_data, path=data_path, transform=transform)
val_que = CVUSA_Dataset_Eval(data_folder=data_path, split='val', img_type='query', transforms=transform)
val_ref = CVUSA_Dataset_Eval(data_folder=data_path, split='val', img_type='reference', transforms=transform)





def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    embed_dim = 1000
    lr = 0.00001
    batch_size = 64
    epochs = 100
    expID = get_rand_id()
    loss_margin = 1


    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    val_loader_que = DataLoader(val_que, batch_size=batch_size, shuffle=False)
    val_loader_ref = DataLoader(val_ref, batch_size=batch_size, shuffle=False)

    os.mkdir(f'model_weights/{expID}')

    # model = ResNet(emb_dim=embed_dim).to(device)
    # model_r = ResNet(emb_dim=embed_dim).to(device)
    # model_q = ResNet(emb_dim=embed_dim).to(device)

    # model = ResNet().to(device)
    # model = VIT().to(device)
    model = CLIP_model(embed_dim=embed_dim)
    
    # torch.save(model, f'model_weights/{expID}/model_st.pth')

    # criterion = TripletLoss(margin=loss_margin)
    # criterion = nn.TripletMarginLoss(margin=0.5)
  
    # criterion = SoftTripletBiLoss()

    loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion = InfoNCE(loss_function=loss_fn,
                            device=device,
                            )



    parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print(name)
    optimizer = optim.Adam(parameters, lr=lr)
    # optimizer = optim.AdamW(parameters, lr=lr)
    # optimizer = optim.SGD(parameters, lr=lr)

    
    
    hyparam_info(emb_dim=embed_dim, loss_id=expID, ln_rate=lr, batch=batch_size, epc=epochs, ls_mrgn=loss_margin, trn_sz=train_data.shape[0], mdl_nm=model.modelName)

    print("Training Start")
    all_loses = train(model, criterion, optimizer, train_loader, num_epochs=epochs, dev=device)
    df_loss = pd.DataFrame({'Loss': all_loses})
    df_loss.to_csv(f'losses/losses_{expID}.csv')



    print("\nExtract Features:")
    query_features, query_labels = predict(model=model, dataloader=val_loader_que, dev=device, isQuery=True)
    reference_features, reference_labels = predict(model = model, dataloader=val_loader_ref, dev=device, isQuery=False) 
    


    print("Compute Scores:")
    # r1 =  calculate_scores(query_features, reference_features, query_labels, reference_labels, step_size=1000, ranks=[1, 5, 10])
    r1 =  accuracy(query_features=query_features, reference_features=reference_features, query_labels=query_labels, topk=[1, 5, 10])
    
    save_losses(df=df_loss, 
                emb_dim=embed_dim, 
                loss_id=expID, 
                ln_rate=lr, 
                batch=batch_size, 
                epc=epochs, 
                ls_mrgn=loss_margin, 
                trn_sz=train_data.shape[0],
                mdl_nm=model.modelName,
                rslt=r1)


    print(r1) 
        

    torch.save(model, f'model_weights/{expID}/model_tr.pth')






if __name__ == '__main__':
    main()


/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Device: cuda


/home/fahimul/.conda/envs/CVGL/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Hyperparameter info:
Exp ID: 6318896
Embedded dimension: 1000
Learning rate: 1e-05
Batch Size: 64
Loss Margin: 1
Epoch: 100
Training Size: 10658
Model Name: CLIP


Training Start

Date: 2024-05-21 15:14:57.089868

Epoch#0


100%|██████████| 167/167 [04:22<00:00,  1.57s/it]


...

Epoch: 99/100 Loss: 1.9749112129211426
Epoch#99


100%|██████████| 167/167 [03:59<00:00,  1.44s/it]


Epoch: 100/100 Loss: 2.039949655532837

Date: 2024-05-21 21:56:50.592842


Extract Features:


100%|██████████| 139/139 [01:08<00:00,  2.04it/s]


Compute Scores:
Percentage-top1:0.7203962179198559, top5:2.8590724898694284, top10:5.481764970733904, top1%:25.529040972534894, time:0.2287003993988037
[ 0.72039622  2.85907249  5.48176497 25.52904097]
